### Convolution neural networks

In [14]:
# importer
import torch
import numpy as np
import sys
import os
from torch.nn import Module, Conv2d, Linear, AvgPool2d, Sigmoid, LogSoftmax, CrossEntropyLoss
from torch.optim import Adam
from torch import flatten
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
sys.path.append(os.path.abspath(".."))
from ml_utils import vizualization, parse_config, activation_decode

In [15]:
VALID_JSON={
    "nn_config":
    {
        "type":"object",
        "properties":
        {
            "act_f":
            {
                "type":"string",
                "enum":["sig", "silu", "relu","identity"]
            },
            "lr":
            {
                "type":"number",
                "maximum":1.0
            },
            "architecture":
            {
                "type":"string",
                "enum":["lenet", "alexnet"]
            },
            "epochs":
            {
                "type":"integer",
                "minimum":10
            },
            "batch_size":
            {
                "type":"integer",
                "minimum":10
            }
        },
        "required":["architecture", "epochs", "batch_size", "lr"]
    },
    "data":
    {
        "type":"object",
        "properties":
        {
            "x_normal":
            {
                "type":"number",
                "minimum":0.001,
                "maximum":1.0
            },
            "y_norm":
            {
                "type":"number",
                "minimum":0.001,
                "maximum":1.0
            },
            "dataset":
            {
                "type":
                {
                    "string"
                },
                "enum":["MNIST", "KMNIST"]
            }
        }
    }
}

### Model
![Reference](img\alexnet.png)

In [16]:
class LeNet(Module):
    def __init__(self, act_f_str:str) -> None:
        super().__init__()
        self.conv_l_1 = Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.act_1 = activation_decode(act_f_str)
        self.avg_pool_1 = AvgPool2d(kernel_size=2, stride=2)
        self.conv_l_2 = Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.act_2 = activation_decode(act_f_str)
        self.avg_pool_2 = AvgPool2d(kernel_size=2, stride=2)
        self.fullCon1 = Linear(in_features=16*4*4, out_features=120)
        self.act_3 = activation_decode(act_f_str)
        self.fullCon2 = Linear(in_features=120, out_features=84)
        self.act_4 = activation_decode(act_f_str)
        self.fullCon3 = Linear(in_features=84, out_features=10)
    
    def forward(self, x):
        # conv and pool
        x = self.avg_pool_1(self.act_1(self.conv_l_1(x)))
        x = self.avg_pool_2(self.act_2(self.conv_l_2(x)))
        
        # transform
        x = flatten(x, 1)

        # full connection
        x = self.act_3(self.fullCon1(x))
        x = self.act_4(self.fullCon2(x))
        x =  LogSoftmax(dim=1)(self.fullCon3(x))

        return x

class AlexNet():
    def __init__(self, act_f_str:str) -> None:
        pass

### Prepare data for learning

In [17]:
def get_data(config:dict, datasrc:object, train_path:str, test_path:str):
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((config['data']['x_norm']), (config['data']['y_norm']))]
    )#(0.1307), (0.3081)
    return  DataLoader(
        dataset=datasrc(root=train_path, train=True, transform=transform, download=True),
        batch_size=config['nn_config']['batch_size'], 
        shuffle=True,
        num_workers=4), DataLoader(
        dataset=datasrc(root=test_path, train=False, transform=transform, download=True),
        batch_size=config['nn_config']['batch_size'], 
        shuffle=True,
        num_workers=4) 

In [18]:
def train(device:torch.device, model:object, config:dict, train_loader:DataLoader, criterion:CrossEntropyLoss, optimizer:Adam):
    model.train()
    x_epoch = np.arange(0, config['nn_config']['epochs'])
    y_loss = np.zeros(config['nn_config']['epochs'])

    for epoch in range(config['nn_config']['epochs']):
        running_loss = 0.0
        loss = None
        for (data, target) in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        y_loss[epoch] = running_loss/len(train_loader)
        print(f'Epoch {epoch+1} finished with avg loss: {running_loss/len(train_loader):.4f}')
    
    vizualization(x_epoch, 'epochs', y_loss, 'loss', 'loss over epochs')

def test(device:torch.device, model:object, test_loader:DataLoader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predict = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predict==target).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')


def learn(config:dict)->None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    train_loader, test_loader = get_data(config, datasets.MNIST if config['data']['dataset'] else datasets.KMNIST, 'data/train', 'data/test')
    model = LeNet(config['nn_config']['act_f']).to(device) if config['nn_config']['architecture'] == 'lenet' else AlexNet(config['nn_config']['act_f']).to(device)
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), config['nn_config']['lr'])
    train(device, model, config, train_loader, criterion, optimizer)
    test(device, model, test_loader)

In [ ]:
torch.backends.cudnn.benchmark = True
config = parse_config('config/config1.json', VALID_JSON)
learn(config)

In [ ]:
config = parse_config('config/config1.json', VALID_JSON)
learn(config)